In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
# Hyper-parameters
input_size = 1
output_size = 1
num_epochs = 60
learning_rate = 0.001

# Toy dataset
x_train = np.array([[3.3], [4.4], [5.5], [6.71], [6.93], [4.168], 
                    [9.779], [6.182], [7.59], [2.167], [7.042], 
                    [10.791], [5.313], [7.997], [3.1]], dtype=np.float32)

y_train = np.array([[1.7], [2.76], [2.09], [3.19], [1.694], [1.573], 
                    [3.366], [2.596], [2.53], [1.221], [2.827], 
                    [3.465], [1.65], [2.904], [1.3]], dtype=np.float32)

# Linear regression model
model = nn.Linear(input_size, output_size)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
for epoch in range(num_epochs):
    # Convert numpy arrays to torch tensors
    inputs = torch.from_numpy(x_train)
    targets = torch.from_numpy(y_train)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(
            epoch+1, num_epochs, loss.item()))

Epoch [5/60], Loss: 1.7186
Epoch [10/60], Loss: 1.0108
Epoch [15/60], Loss: 0.7235
Epoch [20/60], Loss: 0.6066
Epoch [25/60], Loss: 0.5586
Epoch [30/60], Loss: 0.5387
Epoch [35/60], Loss: 0.5301
Epoch [40/60], Loss: 0.5260
Epoch [45/60], Loss: 0.5238
Epoch [50/60], Loss: 0.5224
Epoch [55/60], Loss: 0.5213
Epoch [60/60], Loss: 0.5203


In [3]:

# Create a directory for the model
%mkdir -p /tmp/pytorch

In [4]:
# Save the model to a file and remove it, we will reload it in the next step
torch.save(model.state_dict(), "/tmp/pytorch/model.pt")
print("Model saved")
del(model)
# Let's check if the model variable is really gone
try: 
    model
except NameError:
    print("Model variable removal successful")

Model saved
Model variable removal successful


In [5]:
# Load the model from the file, remember you must define the model type first 
# (for example nn.Linear)
model = nn.Linear(input_size, output_size)
model.load_state_dict(torch.load("/tmp/pytorch/model.pt"))
# Check model type
model.eval()

Linear(in_features=1, out_features=1, bias=True)

In [6]:
# Predict function needs the model variable and inputs
def predict(model, inputs):
    pred = torch.from_numpy(np.array(inputs, dtype=np.float32))
    pred = model(pred)
    pred = pred.data.numpy()
    return [str(x) for x in pred]
# Test on 2 values
predict(model, [[2.0], [4.0]])

['[0.09903026]', '[1.0874232]']

In [7]:
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers.pytorch import deploy_pytorch_model
clipper_conn = ClipperConnection(DockerContainerManager())

In [8]:
clipper_conn.connect()

20-11-25:15:11:21 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at localhost:1337


In [9]:
# You can see some Clipper containers by simply running:
!docker ps

CONTAINER ID        IMAGE                               COMMAND                  CREATED             STATUS              PORTS                                            NAMES
12d83d7e1899        prom/prometheus:v2.9.2              "/bin/prometheus --c…"   12 minutes ago      Up 12 minutes       0.0.0.0:9090->9090/tcp                           metric_frontend-98252
cbbc7314c342        clipper/frontend-exporter:0.4.1     "python /usr/src/app…"   12 minutes ago      Up 12 minutes                                                        query_frontend_exporter-32261
d1b44758e60a        clipper/query_frontend:0.4.1        "/clipper/query_fron…"   12 minutes ago      Up 12 minutes       0.0.0.0:1337->1337/tcp, 0.0.0.0:7000->7000/tcp   query_frontend-32261
d1d5e3786170        clipper/management_frontend:0.4.1   "/clipper/mgmt_front…"   12 minutes ago      Up 12 minutes       0.0.0.0:1338->1338/tcp                           mgmt_frontend-25215
5cc95c5853bc        redis:alpine                   

In [10]:
# List all applications
clipper_conn.get_all_apps()

['nmt']

In [11]:
# List all models
clipper_conn.get_all_models()

['nmtfairseq:1']

In [12]:
clipper_conn.register_application(name="pytorch-app", input_type="doubles", default_output="-1.0", slo_micros=100000)

20-11-25:15:12:50 INFO     [clipper_admin.py:236] [default-cluster] Application pytorch-app was successfully registered


In [24]:
# "?deploy_pytorch_model" to let the notebook show you the definition of the method
deploy_pytorch_model(
    clipper_conn,
    name="pytorch-mod",
    version=1,
    input_type="doubles",
    func=predict,
    pytorch_model=model)

20-11-25:15:15:31 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpeqx6yqrtclipper
20-11-25:15:15:31 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-11-25:15:15:31 INFO     [pytorch.py:204] Torch model saved
20-11-25:15:15:31 INFO     [pytorch.py:218] Using Python 3.6 base image
20-11-25:15:15:31 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpeqx6yqrtclipper
20-11-25:15:15:31 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/pytorch36-container:0.4.1
20-11-25:15:15:31 INFO     [clipper_admin.py:539] [default-cluster]  ---> e3c73c7ad6b9
20-11-25:15:15:31 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmpeqx6yqrtclipper /model/
20-11-25:15:15:31 INFO     [clipper_admin.py:539] [default-cluster]  ---> c95d5f6c09db
20-11-25:15:15:31 INFO     [clipper_admin.py:539] [default-cluster] Successfully built c95d5f6c09db
20-11-25:15:15:31 INFO     [clipp

In [25]:
clipper_conn.link_model_to_app(
    app_name="pytorch-app",
    model_name="pytorch-mod")

20-11-25:15:15:35 INFO     [clipper_admin.py:303] [default-cluster] Model pytorch-mod is now linked to application pytorch-app


In [28]:
clipper_conn.get_all_apps(), clipper_conn.get_all_models(),clipper_conn.get_linked_models(app_name="pytorch-app")

(['nmt', 'pytorch-app'], ['nmtfairseq:1', 'pytorch-mod:1'], ['pytorch-mod'])

In [29]:
# Get query address
query_address = clipper_conn.get_query_addr()
print(query_address)

localhost:1337


In [30]:
# Run a query
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://"+query_address+"/pytorch-app/predict", headers=headers, data=json.dumps({
    "input": [2.0]})).json()
# The result is the same as in the local prediction!

{'query_id': 3, 'output': [0.09903026], 'default': False}

In [18]:
clipper_conn.unlink_model_from_app(model_name="pytorch-mod", app_name="pytorch-app")

20-11-25:15:13:36 INFO     [clipper_admin.py:344] Model pytorch-mod is now removed to application pytorch-app


In [19]:
clipper_conn.stop_models('pytorch-mod')

20-11-25:15:13:52 INFO     [clipper_admin.py:1285] Model pytorch-mod:1 was successfully deleted
20-11-25:15:13:52 INFO     [clipper_admin.py:1319] [default-cluster] Stopped all containers for these models and versions:
{'pytorch-mod': ['1']}
